In [ ]:
import sys
sys.path.append("..")
import os
import math
import json
import parsers
import random
import torch
import re
import xml
from collections import defaultdict
from transformers import AutoTokenizer
from constants import SPECIAL_TOKENS
from config import Config
from IPython.display import display, Markdown

In [ ]:
TEST_RES_FILEPATH = 'results/best-checkpoint/test.out.json'
TRAIN_RES_FILEPATH = 'results/best-checkpoint/train.out.json'
CKPT_PATH = 'best-checkpoint.mdl'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_results(model_dir, res_filepath):
    print(f'Loading results from {model_dir}')
    saved_dict = torch.load(os.path.join(model_dir, CKPT_PATH), map_location = DEVICE)
    config = Config.from_dict(saved_dict['config'])
    
    tokenizer = AutoTokenizer.from_pretrained(config.bert_model_name)
    tokenizer.add_tokens(SPECIAL_TOKENS)

    # model = TextMappingModel(config)
    # model.load_bert(config.bert_model_name)
    # model.bert.resize_token_embeddings(len(tokenizer))
    # model.load_state_dict(saved_dict['model'])
    # model.to(DEVICE)
    
    # print(f'Loading dataset for {model_dir}')
    # dataset = LPMappingDataset(
    #     path = '../data/test.jsonl',
    #     tokenizer = tokenizer,
    #     max_length = config.max_length,
    #     gpu = torch.cuda.is_available(),
    #     enrich_ner = config.enrich_ner,
    # )
    # dataset.numberize()
    # dataloader = DataLoader(dataset, batch_size = 1, shuffle = False, collate_fn = dataset.collate_fn)
    
    with open(os.path.join(model_dir, res_filepath)) as f:
        results = json.load(f)

    return results, config, tokenizer

In [ ]:
model_dir = '../output/default/20230514_151250415'

results, config, tokenizer = load_results(model_dir, TEST_RES_FILEPATH)

In [ ]:
def parse_convert(output, order_mapping):
    parser = parsers.ModelOutputXMLParser(print_errors = True)
    return parser.parse(output, order_mapping)


def convert_to_canonical(parsed, is_gold: bool):
    return parsers.convert_to_canonical(parsed, is_gold = is_gold)


def print_diff(gp):
    gold = gp['gold']
    pred = gp['pred']
    doc = re.sub('<[^<]+>', '', gp['document'])
    doc_id = gp['doc_id']
    
    gold_canonical = convert_to_canonical(parse_convert(gold, gp['order_mapping']), True)
    pred_canonical = convert_to_canonical(parse_convert(pred, gp['order_mapping']), False)
    
    text = f'**Document** ({doc_id})\n\n```\n{doc}\n```\n\n'
    text += f'**Gold**\n\n```xml\n{gold}\n```\n\n'
    text += f'**Pred**\n\n```xml\n{pred}\n```\n\n'
    text += f'**Gold Canonical**\n\n```xml\n{gold_canonical}\n```\n\n'
    text += f'**Pred Canonical**\n\n```xml\n{pred_canonical}\n```\n\n'
    
    display(Markdown(text))

In [ ]:
err_gp_pairs = []
for gp in results['gold_pred_pairs']:
    gold = gp['gold']
    pred = gp['pred']
    acc = gp['accuracy']
    if acc < 1:
        err_gp_pairs.append(gp)

err_gp_pairs_it = iter(err_gp_pairs)

In [ ]:
example = next(err_gp_pairs_it)
print_diff(example)

### Check Training Examples

This is to validate how many examples have the objective defined on a subset of variables.

In [ ]:
train_results, train_config, train_tokenizer = load_results(model_dir, TRAIN_RES_FILEPATH)

In [ ]:
for gp in train_results['gold_pred_pairs']:
    gold = gp['gold']
    gold_canonical = convert_to_canonical(parse_convert(gold, gp['order_mapping']), True)
    
    for obj_param in gold_canonical.objective:
        if math.isclose(obj_param, 0):
            print(f'Objective is defined on a subset of variables for {gp["doc_id"]}. Objective: {gold_canonical.objective}')
            continue

### Check Constraint Directions of Correct Test Predictions

The distribution of operators in the constraints of the correctly predicted test examples.

In [ ]:
test_results, test_config, test_tokenizer = load_results(model_dir, TEST_RES_FILEPATH)

In [ ]:
parser = parsers.ModelOutputXMLParser()
const_type_dir_count = defaultdict(lambda : defaultdict(int))

for gp in test_results['gold_pred_pairs']:
    if not math.isclose(gp['accuracy'], 1):
        continue
        
    gold = gp['gold']
    xmltree = parser.xmltree(gold)
    declarations = xmltree.iter('DECLARATION')
    
    for declaration in declarations:
        if declaration.find('OBJ_DIR') is not None:
            continue
            
        const_dir = declaration.find('OPERATOR')
        const_type = declaration.find('CONST_TYPE')
        const_type_dir_count[const_type.text.strip()][const_dir.text.strip()] += 1

In [ ]:
print(json.dumps(const_type_dir_count, indent = 4))

The distribution of operators for different constraint type in the train set.

In [ ]:
with open('../data/train.jsonl') as f:
    train_examples = [json.loads(l) for l in f]
    

const_type_dir_count = defaultdict(lambda : defaultdict(int))

for example in train_examples:
    assert len(example.values()) == 1
    example = next(iter(example.values()))
    for declr in example['const_declarations']:
        const_type_dir_count[declr['type']][declr['operator']] += 1


In [ ]:
print(json.dumps(const_type_dir_count, indent = 4))